In [21]:
import pandas as pd
import os
from math import sqrt,pow

In [ ]:
artists = pd.read_csv(os.path.join('..','data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join('..','data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join('..','data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join('..','data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join('..','data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join('..','data','user_taggedartists.dat'), delimiter='\t')

In [6]:
colnames = ['id','name']
artists2 = artists[colnames]
artists2.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


## Investigating user-user interactions

In [ ]:
def get_friends(userID):
    friends = user_friends.loc[user_friends['userID'] == userID]
    return [x for x in friends['friendID']]

def get_tags(userID):
    tags = user_taggedartists.loc[user_taggedartists['userID'] == userID]
    return [x for x in tags['tagID']]

In [23]:
# Taken from https://dev.to/josethz00/math-for-devs-cosine-similarity-in-python-2524

def cosine_similarity(vector1: list[float], vector2: list[float]) -> float:
    """Returns the cosine of the angle between two vectors."""
    # the cosine similarity between two vectors is the dot product of the two vectors divided by the magnitude of each vector

    dot_product = 0
    magnitude_vector1 = 0
    magnitude_vector2 = 0

    vector1_length = len(vector1)
    vector2_length = len(vector2)

    if vector1_length > vector2_length:
        # fill vector2 with 0s until it is the same length as vector1 (required for dot product)
        vector2 = vector2 + [0] * (vector1_length - vector2_length)
    elif vector2_length > vector1_length:
        # fill vector1 with 0s until it is the same length as vector2 (required for dot product)
        vector1 = vector1 + [0] * (vector2_length - vector1_length)

    # dot product calculation
    for i in range(len(vector1)):
        dot_product += vector1[i] * vector2[i]

    # vector1 magnitude calculation
    for i in range(len(vector1)):
        magnitude_vector1 += pow(vector1[i], 2)

    # vector2 magnitude calculation
    for i in range(len(vector2)):
        magnitude_vector2 += pow(vector2[i], 2)

    # final magnitude calculation
    magnitude = sqrt(magnitude_vector1) * sqrt(magnitude_vector2)

    # return cosine similarity
    return dot_product / magnitude

In [28]:
def friends_tag_similarity(userID):
    friends = get_friends(userID)
    sims = []
    for friend in friends:
        sims.append(cosine_similarity(get_tags(2),get_tags(friend)))
    return sum(sims)/len(sims)

In [35]:
sims2 = []
for userID in user_artists['userID'].unique():
    sims2.append(friends_tag_similarity(userID))
print(sum(sims2)/len(sims2))

0.30130843520311024
